In [14]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [15]:
import ipywidgets as widgets
from IPython.display import display
import json
from beta import Artifact

def on_file_selected(change):
    if change['type'] == 'change' and change['name'] == 'value':
        file_content = change['new']
        if not file_content:
            print("No file selected")
            return
        
        print("File content type:", type(file_content))
        print("File content:", file_content)
        
        # Handle the tuple structure
        if isinstance(file_content, tuple) and len(file_content) > 0:
            file_data = file_content[0]
            file_name = file_data['name']
            
            print(f"Selected file: {file_name}")
            print("File data type:", type(file_data))
            
            try:
                # Create the Artifact
                artifact = Artifact(files=[file_name])
                print("Artifact created successfully")
                print("Artifact DataFrame:")
                artifact.df.collect().show()
            except Exception as e:
                print(f"Error creating Artifact: {str(e)}")
        else:
            print("Unexpected file content format")

# Create a file upload widget
file_upload = widgets.FileUpload(
    accept='',  # Accept all file types
    multiple=False  # Allow only single file selection
)

# Display the file upload widget
display(file_upload)

# Attach the callback to the file upload widget
file_upload.observe(on_file_selected, names='value')

FileUpload(value=(), description='Upload')

In [16]:
from beta.agents import Agent
from beta.models.obj.model_object import VLLM
from vllm import LLM
from beta.tools.obj import CalculatorTool, DataFrameTool
from beta.models.serve.engines import VLLMEngine
from beta.models.serve.engines import OpenAIEngineConfig
from mlflow.client import MlflowClient
from daft import DataFrame
from beta.models.serve.engines.openai_engine import OpenAIEngine

In [17]:
!pip install mlflow


In [18]:
import mlflow
mlflow.set_experiment("Dratos AI")
with mlflow.start_run():
    mlflow.log_param("model", "gpt-4o")
    mlflow.log_param("temperature", 0.5)
    mlflow.log_param("max_tokens", 4096)
    mlflow.log_param("top_p", 1)
    mlflow.log_param("frequency_penalty", 0)
    mlflow.log_param("presence_penalty", 0)
    prompt = "What is the capital of France?"
    mlflow.log_param("prompt", prompt)

    config = OpenAIEngineConfig(data={
        "api_key": "sk-svcacct-0XzynrHXZP8dxo4KmugT66xjiOUdjUZiK5MxwazdlN2inmbMfHSVExqV3TSRevTjlaBT3BlbkFJbRpZXPpwRrewrGKzTzdGi1oBesleibiYSz9kklAt5uuUwH3FCbiCgATMeIX4q5-4RAA",
        "temperature": 0.5,
        "max_tokens": 4096,
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0
    })
    engine = OpenAIEngine(model_name="gpt-4o", config=config)
    response = await engine.generate_structured(prompt, structure="{city: str, capital: str}")

print(response)


INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0
INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
  "city": "Paris",
  "capital": "France"
}


In [19]:
from beta.prompts import prompt

@prompt
def my_prompt(question):
    """{{ question }}"""

@prompt
def cot(my_prompt, max_steps):
    """
    System: 
        Using Chain of Thought:
        max_steps = {{max_steps}}

        for step in max_steps:
            Reason about the request and append, "Therefore"
        
        Make a concluding response 

    User:
    {{ my_prompt }}
    """

In [20]:
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import date
from ray import serve
@serve.deployment(name="Test0")
class QuarterlyEarnings(BaseModel):
    company_name: str = Field(..., description="Name of the company")
    fiscal_quarter: int = Field(..., ge=1, le=4, description="Fiscal quarter (1-4)")
    fiscal_year: int = Field(..., description="Fiscal year")
    revenue: float = Field(..., description="Total revenue for the quarter")
    net_income: float = Field(..., description="Net income for the quarter")
    earnings_per_share: float = Field(..., description="Earnings per share")
    report_date: date = Field(..., description="Date of the earnings report")
    analyst_expectations_met: Optional[bool] = Field(None, description="Whether analyst expectations were met")
    key_highlights: Optional[List[str]] = Field(None, description="Key highlights from the earnings report")

    def __call__(self, *args, **kwargs):
        return "QuarterlyEarnings deployment is working"


In [21]:
import ray
from beta.models.serve.engines.openai_engine import OpenAIEngine

def openai_engine_serializer(engine):
    # Return the arguments needed to reconstruct the engine
    return (engine.model_name, engine.config)

def openai_engine_deserializer(model_name, config):
    # Reconstruct the engine
    return OpenAIEngine(model_name=model_name, config=config)

# Register the custom serializer with Ray
ray.util.register_serializer(
    OpenAIEngine,
    serializer=openai_engine_serializer,
    deserializer=openai_engine_deserializer
)

In [22]:
from datetime import datetime
import inspect
from pydantic import BaseModel
import daft
from ray import serve
from typing import get_origin, get_args

def pydantic_to_daft_schema(model_class):
    if isinstance(model_class, serve.Deployment):
        # This is a Ray Serve deployment
        model_class = model_class.func_or_class

    if not inspect.isclass(model_class) or not issubclass(model_class, BaseModel):
        raise ValueError("Input must be a Pydantic model class or a Ray Serve deployment of a Pydantic model")

    field_dict = {}
    for name, field in model_class.model_fields.items():
        if field.annotation == str:
            field_dict[name] = daft.DataType.string()
        elif field.annotation == int:
            field_dict[name] = daft.DataType.int64()
        elif field.annotation == float:
            field_dict[name] = daft.DataType.float64()
        elif field.annotation == bool:
            field_dict[name] = daft.DataType.boolean()
        elif field.annotation == datetime.date:
            field_dict[name] = daft.DataType.date()
        elif get_origin(field.annotation) == list and get_args(field.annotation)[0] == str:
            field_dict[name] = daft.DataType.list(daft.DataType.string())
        else:
            field_dict[name] = daft.DataType.python()  # Fallback for complex types
    return daft.DataType.struct(field_dict)

In [23]:
from daft import Schema
import daft
import json
import ray
from ray import serve
from beta.agents.agent import Agent, Metadata, SchemaWrapper
from beta.tools.obj.calculator_tool import CalculatorTool
from beta.tools.obj.dataframe_tool import DataFrameTool
from beta.models.serve.engines.openai_engine import OpenAIEngine, OpenAIEngineConfig

# Initialize Ray and Serve
ray.init(ignore_reinit_error=True)
serve.start()

# Create OpenAIEngine instances
config = OpenAIEngineConfig(data={
    "api_key": "sk-svcacct-0XzynrHXZP8dxo4KmugT66xjiOUdjUZiK5MxwazdlN2inmbMfHSVExqV3TSRevTjlaBT3BlbkFJbRpZXPpwRrewrGKzTzdGi1oBesleibiYSz9kklAt5uuUwH3FCbiCgATMeIX4q5-4RAA",
    "temperature": 0.5,
    "max_tokens": 4096,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
})  
model_engine = OpenAIEngine(model_name="gpt-4o", config=config)
embedding_engine = OpenAIEngine(model_name="gpt-4o", config=config)
stt_engine = OpenAIEngine(model_name="gpt-4o", config=config)

# QuarterlyEarnings as a DataFrame Schema
QuarterlyEarningsSchema = pydantic_to_daft_schema(QuarterlyEarnings)

# Create the Agent
q1_agent = Agent(
    name="Q1 Agent",
    model=model_engine,
    embedding=embedding_engine,
    stt=stt_engine,
    tools=[CalculatorTool, DataFrameTool],
    metadata=Metadata(schema=SchemaWrapper(schema=QuarterlyEarningsSchema)),
    engine=model_engine,
    is_async=True
)

from beta.prompts import prompt

@prompt
def my_prompt(question):
    """What were the key highlights of the Q4 2024 earnings?"""

response = await q1_agent.process(prompt=my_prompt())
print(response)


2024-09-20 14:58:43,852	INFO worker.py:1619 -- Calling ray.init() again after it has already been called.
INFO 2024-09-20 14:58:43,858 serve 2064130 api.py:259 - Connecting to existing Serve app in namespace "serve". New http options will not be applied.
INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0
INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0
INFO:root:Agent Q1 Agent called with prompt: What were the key highlights of the Q4 2024 earnings?
INFO:root:Agent status set to processing
INFO:root:Sending request to model
INFO:root:Prompt: What were the key highlights of the Q4 2024 earnings?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Awaiting model response
INFO:root:Model response received: I currently do not have access to real-t

I currently do not have access to real-time data or specific earnings reports for Q4 2024. For detailed and accurate highlights of a company's Q4 2024 earnings report, I recommend checking the company's official press release, investor relations website, or financial news websites and databases. Typically, key highlights in an earnings report would include:

1. **Revenue**: Total revenue generated during the quarter.
2. **Net Income**: Profit after all expenses have been deducted from total revenue.
3. **Earnings Per Share (EPS)**: Amount of profit attributed to each outstanding share of common stock.
4. **Gross Margin**: Revenue minus the cost of goods sold, expressed as a percentage.
5. **Operating Income**: Earnings before interest and taxes (EBIT).
6. **Cash Flow**: Net cash generated from operating, investing, and financing activities.
7. **Guidance**: Company’s expectations for future performance.
8. **Key Metrics by Segment**: Performance of different business segments or divisi

In [25]:
# New Cell: Using CalculatorTool and DataFrameTool with Agent

from beta.tools import CalculatorTool, DataFrameTool
from beta.agents import Agent
from beta.prompts import prompt

# Define a new prompt that leverages both tools
@prompt
def compute_and_analyze(question):
    """
    You are an intelligent assistant equipped with Calculator and DataFrame tools.
    
    Task:
    - Calculate the total revenue for Q4 2024.
    - Provide a summary of key highlights in a table format.
    
    User Question:
    {{ question }}
    """

# Initialize the Agent with CalculatorTool and DataFrameTool
compute_analyze_agent = Agent(
    name="ComputeAnalyzeAgent",
    model=model_engine,          # Ensure model_engine is defined in your environment
    embedding=embedding_engine,  # Ensure embedding_engine is defined in your environment
    stt=stt_engine,              # Ensure stt_engine is defined in your environment
    tools=[CalculatorTool, DataFrameTool],
    metadata=Metadata(schema=SchemaWrapper(schema=QuarterlyEarningsSchema)),
    engine=model_engine,
    is_async=True
)

# Define the question to be processed
question = "Calculate the total revenue for Q4 2024 and provide a summary of key highlights in a table."

# Debugging: Print the constructed prompt before processing
constructed_prompt = compute_and_analyze(question)
print("Constructed Prompt:")
print(constructed_prompt)

# Process the prompt with the agent
response = await compute_analyze_agent.process(prompt=constructed_prompt)

# Display the response
print("Agent Response:")
print(response)

INFO:root:Agent ComputeAnalyzeAgent called with prompt: You are an intelligent assistant equipped with Calculator and DataFrame tools.
    
    Task:
    - Calculate the total revenue for Q4 2024.
    - Provide a summary of key highlights in a table format.
    
    User Question:
INFO:root:Agent status set to processing
INFO:root:Sending request to model
INFO:root:Prompt: You are an intelligent assistant equipped with Calculator and DataFrame tools.
    
    Task:
    - Calculate the total revenue for Q4 2024.
    - Provide a summary of key highlights in a table format.
    
    User Question:


Constructed Prompt:
You are an intelligent assistant equipped with Calculator and DataFrame tools.
    
    Task:
    - Calculate the total revenue for Q4 2024.
    - Provide a summary of key highlights in a table format.
    
    User Question:


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Awaiting model response
INFO:root:Model response received: To calculate the total revenue for Q4 2024 and provide a summary of key highlights in a table format, I will need the revenue data for that period. Please provide the revenue figures for each month in Q4 2024 (October, November, and December) or a data set containing this information.
INFO:root:Agent status set to idle


Agent Response:
To calculate the total revenue for Q4 2024 and provide a summary of key highlights in a table format, I will need the revenue data for that period. Please provide the revenue figures for each month in Q4 2024 (October, November, and December) or a data set containing this information.


In [ ]:
!pip install returns

In [ ]:
from returns.future import Future

async def process_agent_response(prompt):
    return Future.from_sync(lambda: q1_agent.process(my_prompt(prompt)))

response = await process_agent_response("What were the key highlights of the Q4 2024 earnings?").close()
print(response)

TypeError: object NoneType can't be used in 'await' expression

In [ ]:
mlflow_client = MlflowClient("http://localhost:5000")
config = OpenAIEngineConfig(data={"temperature": 0.5, "max_tokens": 4096, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0})
llm = LLM(model="meta-llama/Meta-Llama-3.1-8B-Instruct")
vllm = VLLM(model=LLM)
engine = VLLMEngine(model_name="meta-llama/Meta-Llama-3.1-8B-Instruct", mlflow_client=mlflow_client, config=config, api_key="EMPTY", base_url="http://localhost:8000")

paligemma = vllm(
    model_name="google/paligemma-3b-mix-448",
    engine=engine,
)

INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0
INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0


WARNING 09-20 14:06:34 arg_utils.py:839] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 09-20 14:06:34 config.py:911] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 09-20 14:06:34 llm_engine.py:184] Initializing an LLM engine (v0.5.5) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_p

NotImplementedError: 

In [ ]:
from beta.models.obj import llm
from beta.models.serve.engines import LitServeEngine, OpenAIEngine, OpenRouterEngine, VLLMEngine


completion_settings = llm.settings(
    temperature=0.5,
    max_tokens=4096,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

# Defaults to OpenRouter
# Expects OPENROUTER_API_KEY to be defined in .env
gemini_free = llm(
    model_name="google/gemini-pro-1.5-flash",
    settings=completion_settings,
)

# If OPENAI_API_KEY, uses { OpenAi | AsyncOpenAI } clients instead of OpenRouter. 
gpt4o = llm(
    model_name="openai/o1-mini",
    settings=completion_settings,
)

# If ANTHROPIC_API_KEY, uses {Anthropic | AsyncAnthopic }
sonnet = llm(
    model_name="anthopic/claude-3.5-sonnet",
    settings = completion_settings,
)


# Specifying the Engine with OS Model
paligemma = llm(
    model_name="google/paligemma-3b-mix-448",
    engine=LitServeEngine(), 
    completion_settings=completion_settings
)



In [ ]:
from beta.tools import CalculatorTool, DataframeTool

In [ ]:

from beta.agents import BaseAgent, SpeechAgent

artifact_df = daft.DataFrame()
node_df = daft.DataFrame()
document_df = daft.DataFrame()






earnings = Data.Projects.get("Earnings Q4 2024")
knowledge_graph = my_project.Graphs.get("latest")


q1_agent = Agent(
    llm=paligemma,
    tools=[CalculatorTool, DataframeTool],
    context=[earnings],
    memory={'short':[knowl]},
    planning=
    rethinking=
)


@prompt
def my_prompt(question):
    """{{ question}}"""

@prompt
def cot(my_prompt,max_steps):
    """
    System: 
        Using Chain of Thought:
        max_steps = {{max_steps}}

        for step in max_steps:
            Reason about the request and append, "Therefore"
        
        Make a concluding response 

    User:
    {{ my_prompt }}
    """

from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import date

class QuarterlyEarnings(BaseModel):
    """
    Represents quarterly earnings data for a company.
    """
    company_name: str = Field(..., description="Name of the company")
    fiscal_quarter: int = Field(..., ge=1, le=4, description="Fiscal quarter (1-4)")
    fiscal_year: int = Field(..., description="Fiscal year")
    revenue: float = Field(..., description="Total revenue for the quarter")
    net_income: float = Field(..., description="Net income for the quarter")
    earnings_per_share: float = Field(..., description="Earnings per share")
    report_date: date = Field(..., description="Date of the earnings report")
    analyst_expectations_met: Optional[bool] = Field(None, description="Whether analyst expectations were met")
    key_highlights: Optional[List[str]] = Field(None, description="Key highlights from the earnings report")


response = await my_agent(
    prompt=
    response_model=[Address], 
        reponse_model_strict = False
)